In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import requests
import xmltodict

In [ ]:
quarterly_data = pd.read_pickle(
    "/Users/nicolasroever/Documents/Promotion/Debt Crisis/debt_crisis/bld/data/step_one_regression_dataset_output_quarterly.pkl",
)

In [ ]:
quarterly_data.columns

In [ ]:
dictionary = pd.read_csv(
    "/Users/nicolasroever/Documents/Promotion/Debt Crisis/debt_crisis/bld/data/sentiment_data/sentiment_word_count_clean.csv",
)

In [ ]:
dictionary.head()

In [ ]:
# Plot positives

# Filter the DataFrame
filtered_data = dictionary[
    (dictionary["Positive_Indicator"] == 1) & (dictionary["Count"] > 600)
]

# Sort the DataFrame
sorted_data = filtered_data.sort_values(by="Count", ascending=False)

# Create the plot
plt.figure(figsize=(10, 6))
plt.barh(sorted_data["Word"], sorted_data["Count"])
plt.xlabel("Word")
plt.ylabel("Count")
plt.title("Word Counts for Positive Words")
plt.yticks(fontsize=8)  # Rotate x-axis labels for better readability
plt.show()

In [ ]:
# Plot positives

# Filter the DataFrame
filtered_data_2 = dictionary[
    (dictionary["Negative_Indicator"] == 1) & (dictionary["Count"] > 600)
]

# Sort the DataFrame
sorted_data_2 = filtered_data_2.sort_values(by="Count", ascending=False)

# Create the plot
plt.figure(figsize=(10, 6))
plt.barh(sorted_data_2["Word"], sorted_data_2["Count"])
plt.xlabel("Word")
plt.ylabel("Count")
plt.title("Word Counts for Negativev Words")
plt.yticks(fontsize=8)  # Rotate x-axis labels for better readability
plt.show()

In [ ]:
dictionary.sort_values(by="Count", ascending=False)

In [ ]:
dictionary.T

In [ ]:
dictionary.columns = ["Word", "Count"]

In [ ]:
COUNTRIES_UNDER_STUDY = {
    "austria",
    "belgium",
    "bulgaria",
    "croatia",
    "cyprus",
    "czechia",
    "denmark",
    "estonia",
    "finland",
    "france",
    "germany",
    "greece",
    "hungary",
    "ireland",
    "italy",
    "latvia",
    "lithuania",
    "luxembourg",
    "malta",
    "netherlands",
    "poland",
    "portugal",
    "romania",
    "slovakia",
    "slovenia",
    "spain",
    "sweden",
}

In [ ]:
quarterly_data.columns

In [ ]:
quartertly_countries = set(quarterly_data["Country"].unique())

In [ ]:
common_countries = COUNTRIES_UNDER_STUDY.intersection(quartertly_countries)

In [ ]:
common_countries

# Parameters
Find the parameters at https://data-explorer.oecd.org/vis?df[ds]=dsDisseminateFinalDMZ&df[id]=DSD_NAMAIN1%40DF_QNA_EXPENDITURE_USD&df[ag]=OECD.SDD.NAD&df[vs]=1.0&pd=%2C&dq=Q..AUS.S1..B1GQ.....V..&ly[cl]=TIME_PERIOD&to[TIME_PERIOD]=false&lo=5&lom=LASTNPERIODS 

In [ ]:
url = "https://sdmx.oecd.org/public/rest/data/OECD.SDD.TPS,DSD_BOP@DF_BOP,1.0/USA..CA.B..Q.USD_EXC+XDC.N?dimensionAtObservation=AllDimensions"

# Request Data

In [ ]:
r = requests.get(url)

In [ ]:
dictionary_data = xmltodict.parse(r.content)

In [ ]:
dictionary_data

In [ ]:
dictionary_data["message:GenericData"]["message:DataSet"]["generic:Obs"]

In [ ]:
observation_dictionary = dictionary_data["message:GenericData"]["message:DataSet"][
    "generic:Obs"
][0]

In [ ]:
dictionary_data["message:GenericData"]["message:DataSet"]["generic:Obs"][0][
    "generic:ObsValue"
]

In [ ]:
# 1. obskey
obs_key_data = observation_dictionary["generic:ObsKey"]["generic:Value"]
obs_key_dict = {d["@id"]: d["@value"] for d in obs_key_data}

In [ ]:
# Obs value
obs_value_dict = observation_dictionary["generic:ObsValue"]

In [ ]:
# Obs value
obs_attributes_data = observation_dictionary["generic:Attributes"]["generic:Value"]
obs_attributes_dict = {d["@id"]: d["@value"] for d in obs_attributes_data}

In [ ]:
full_observation = {**obs_key_dict, **obs_value_dict, **obs_attributes_dict}

In [ ]:
pd.DataFrame(full_observation, index=[0])

In [ ]:
full_data = pd.DataFrame()

In [ ]:
for i in range(
    len(dictionary_data["message:GenericData"]["message:DataSet"]["generic:Obs"]),
):
    observation_dictionary = dictionary_data["message:GenericData"]["message:DataSet"][
        "generic:Obs"
    ][i]
    # 1. obskey
    obs_key_data = observation_dictionary["generic:ObsKey"]["generic:Value"]
    obs_key_dict = {d["@id"]: d["@value"] for d in obs_key_data}
    # Obs value
    obs_value_dict = observation_dictionary["generic:ObsValue"]
    # ObsAttributes
    obs_attributes_data = observation_dictionary["generic:Attributes"]["generic:Value"]
    obs_attributes_dict = {d["@id"]: d["@value"] for d in obs_attributes_data}

    full_observation = {**obs_key_dict, **obs_value_dict, **obs_attributes_dict}
    full_data = pd.concat([full_data, pd.DataFrame(full_observation, index=[0])])

In [ ]:
full_data.head()

In [ ]:
full_data["COUNTERPART_AREA"].unique()

In [ ]:
# Set out everything about the request in the format specified by the OECD API
data = oecd.data(resource_id="DSD_NAMAIN1").to_pandas()

df = pd.DataFrame(data).reset_index()
df.head()

In [ ]:
df["MEASURE"]

In [ ]:
# Tell pdmx we want OECD data
oecd = pdmx.Request("OECD")
# Set out everything about the request in the format specified by the OECD API
data = oecd.data(
    resource_id="PDB_LV",
    key="GBR+FRA+CAN+ITA+DEU+JPN+USA.T_GDPEMP.CPC/all?startTime=2010",
).to_pandas()

df = pd.DataFrame(data).reset_index()
df.head()

In [ ]:
data.content["OECD.SDD.NAD:DSD_NAMAIN1@DF_QNA_EXPENDITURE_CAPITA(1.0)"]

In [ ]:
pd.read_xml(
    "https://sdmx.oecd.org/public/rest/data/OECD.SDD.NAD,DSD_NAMAIN1@DF_QNA_EXPENDITURE_CAPITA,1.0/Q............?startPeriod=2022-Q4&dimensionAtObservation=AllDimensions",
)